In [59]:
import pandas as pd

df = pd.read_csv("../data/2023_stock_with_features.csv")
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_55,Trend_55
0,2023-03-22,VALE3,82.00,81.68,81.61,82.75,82.11,18085500,1.485137e+09,100221.0,...,0.798373,-0.000028,2.137893e+09,-0.008858,0.993734,0.0,0.990589,3.0,0.916487,27.0
1,2023-03-22,BBDC4,13.14,13.02,12.96,13.24,13.11,29424300,3.859242e+08,100221.0,...,0.331466,0.000524,8.212858e+08,-0.028503,0.995413,1.0,0.978947,2.0,0.939160,25.0
2,2023-03-22,ITUB4,23.95,23.74,23.68,24.25,23.98,36400300,8.730690e+08,100221.0,...,0.298195,0.003986,1.049310e+09,0.003466,0.997060,1.0,1.002618,3.0,0.940543,27.0
3,2023-03-22,PETR4,23.39,23.33,23.04,23.62,23.34,46435600,1.084006e+09,100221.0,...,0.219932,-0.000771,1.650465e+09,0.001804,0.998502,1.0,1.001890,2.0,0.927333,28.0
4,2023-03-22,ABEV3,13.91,13.95,13.89,14.09,13.99,16107100,2.253774e+08,100221.0,...,0.130690,0.003370,4.744414e+08,-0.011620,1.000000,0.0,0.989362,1.0,1.026353,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2023-11-17,ABEV3,13.71,13.60,13.52,13.74,13.56,36788200,4.991464e+08,125062.0,...,0.160997,-0.000423,3.278229e+08,0.006513,0.997433,0.0,1.003690,2.0,1.026133,24.0
821,2023-11-17,BBDC4,15.57,15.43,15.35,15.60,15.45,34493400,5.331879e+08,125062.0,...,0.307034,0.009613,7.900815e+08,0.019693,0.995484,1.0,1.017407,2.0,1.057416,25.0
822,2023-11-17,VALE3,74.23,74.24,74.10,74.65,74.35,24001500,1.784705e+09,125062.0,...,1.553232,0.005820,1.985253e+09,0.027885,1.000944,2.0,1.017265,4.0,1.094689,33.0
823,2023-11-17,PETR4,35.83,36.71,35.82,36.92,36.53,85895400,3.138326e+09,125062.0,...,0.601606,0.008729,1.865863e+09,0.036888,1.016053,1.0,1.026279,3.0,1.059079,31.0


In [60]:
# we're now predicting classification, so tomorrow's value will be part of the analysis
df["tomorrow"] = df["close"].shift(-1)
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_55,Trend_55
0,2023-03-22,VALE3,82.00,81.68,81.61,82.75,82.11,18085500,1.485137e+09,100221.0,...,0.798373,-0.000028,2.137893e+09,-0.008858,0.993734,0.0,0.990589,3.0,0.916487,27.0
1,2023-03-22,BBDC4,13.14,13.02,12.96,13.24,13.11,29424300,3.859242e+08,100221.0,...,0.331466,0.000524,8.212858e+08,-0.028503,0.995413,1.0,0.978947,2.0,0.939160,25.0
2,2023-03-22,ITUB4,23.95,23.74,23.68,24.25,23.98,36400300,8.730690e+08,100221.0,...,0.298195,0.003986,1.049310e+09,0.003466,0.997060,1.0,1.002618,3.0,0.940543,27.0
3,2023-03-22,PETR4,23.39,23.33,23.04,23.62,23.34,46435600,1.084006e+09,100221.0,...,0.219932,-0.000771,1.650465e+09,0.001804,0.998502,1.0,1.001890,2.0,0.927333,28.0
4,2023-03-22,ABEV3,13.91,13.95,13.89,14.09,13.99,16107100,2.253774e+08,100221.0,...,0.130690,0.003370,4.744414e+08,-0.011620,1.000000,0.0,0.989362,1.0,1.026353,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2023-11-17,ABEV3,13.71,13.60,13.52,13.74,13.56,36788200,4.991464e+08,125062.0,...,0.160997,-0.000423,3.278229e+08,0.006513,0.997433,0.0,1.003690,2.0,1.026133,24.0
821,2023-11-17,BBDC4,15.57,15.43,15.35,15.60,15.45,34493400,5.331879e+08,125062.0,...,0.307034,0.009613,7.900815e+08,0.019693,0.995484,1.0,1.017407,2.0,1.057416,25.0
822,2023-11-17,VALE3,74.23,74.24,74.10,74.65,74.35,24001500,1.784705e+09,125062.0,...,1.553232,0.005820,1.985253e+09,0.027885,1.000944,2.0,1.017265,4.0,1.094689,33.0
823,2023-11-17,PETR4,35.83,36.71,35.82,36.92,36.53,85895400,3.138326e+09,125062.0,...,0.601606,0.008729,1.865863e+09,0.036888,1.016053,1.0,1.026279,3.0,1.059079,31.0


In [61]:
# Create the 'tomorrow' column as the next day's closing price
df["tomorrow"] = df["close"].shift(-1)

# Create the target variable: 1 if tomorrow's close is higher than today's close, else 0
df["target"] = (df["tomorrow"] > df["close"]).astype(int)
df


,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_55,Trend_55
0,2023-03-22,VALE3,82.00,81.68,81.61,82.75,82.11,18085500,1.485137e+09,100221.0,...,0.798373,-0.000028,2.137893e+09,-0.008858,0.993734,0.0,0.990589,3.0,0.916487,27.0
1,2023-03-22,BBDC4,13.14,13.02,12.96,13.24,13.11,29424300,3.859242e+08,100221.0,...,0.331466,0.000524,8.212858e+08,-0.028503,0.995413,1.0,0.978947,2.0,0.939160,25.0
2,2023-03-22,ITUB4,23.95,23.74,23.68,24.25,23.98,36400300,8.730690e+08,100221.0,...,0.298195,0.003986,1.049310e+09,0.003466,0.997060,1.0,1.002618,3.0,0.940543,27.0
3,2023-03-22,PETR4,23.39,23.33,23.04,23.62,23.34,46435600,1.084006e+09,100221.0,...,0.219932,-0.000771,1.650465e+09,0.001804,0.998502,1.0,1.001890,2.0,0.927333,28.0
4,2023-03-22,ABEV3,13.91,13.95,13.89,14.09,13.99,16107100,2.253774e+08,100221.0,...,0.130690,0.003370,4.744414e+08,-0.011620,1.000000,0.0,0.989362,1.0,1.026353,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2023-11-17,ABEV3,13.71,13.60,13.52,13.74,13.56,36788200,4.991464e+08,125062.0,...,0.160997,-0.000423,3.278229e+08,0.006513,0.997433,0.0,1.003690,2.0,1.026133,24.0
821,2023-11-17,BBDC4,15.57,15.43,15.35,15.60,15.45,34493400,5.331879e+08,125062.0,...,0.307034,0.009613,7.900815e+08,0.019693,0.995484,1.0,1.017407,2.0,1.057416,25.0
822,2023-11-17,VALE3,74.23,74.24,74.10,74.65,74.35,24001500,1.784705e+09,125062.0,...,1.553232,0.005820,1.985253e+09,0.027885,1.000944,2.0,1.017265,4.0,1.094689,33.0
823,2023-11-17,PETR4,35.83,36.71,35.82,36.92,36.53,85895400,3.138326e+09,125062.0,...,0.601606,0.008729,1.865863e+09,0.036888,1.016053,1.0,1.026279,3.0,1.059079,31.0


In [62]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

train = df.iloc[:-100]
test = df.iloc[-100:]

features = [
    'open', 'close', 'min', 'max', 'avg', 'quantity', 'volume',
    'ibovespa_close', 'day_of_week', 'daily_return', 'price_range', 'volume_per_quantity',
    'rolling_close_5', 'rolling_std_5', 'rolling_return_5', 'momentum_5', 'rolling_volume_5'
]

model.fit(train[features], train["target"])

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,100
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [63]:
print("Data shape:", df.shape)


Data shape: (825, 27)


In [64]:
from sklearn.metrics import precision_score

preds = model.predict(test[features])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds)


0.8571428571428571

The Random Forest model achieved a precision of 0.86, indicating that its “up” predictions were correct about 86% of the time.

However, since financial data is sequential and non-stationary, a single train-test split may not represent the model’s true performance over time. And such a high precision can be result of overfitting, which you'll soon understand as one of the major problems we've had during this project.

To address this, we adopted a walk-forward (backtesting) strategy, where we train on the first year and test on the next, then progressively expand the training set.
This approach ensures that:

- The model only uses past data to predict the future (avoiding data leakage).
- We can evaluate how well it generalizes to new market conditions over time.

In [65]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [66]:
def backtest(data, model, predictors, start=50, step=220):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

In [67]:
predictions = backtest(df, model, features)

In [68]:
predictions["Predictions"].value_counts()

Predictions
1    446
0    329
Name: count, dtype: int64

In [69]:
precision_score(predictions["target"], predictions["Predictions"])

0.6860986547085202

In [70]:
predictions["target"].value_counts() / predictions.shape[0]

target
1    0.516129
0    0.483871
Name: count, dtype: float64

## Backtesting (Walk-Forward Validation)

After introducing **backtesting (walk-forward validation)**, the model evaluation became more realistic and representative of how it would perform in real-world trading scenarios.

### Why Backtesting Matters

* **Chronological Training:**
  Each training period uses only past data to predict the next horizon, avoiding future data leakage.

* **Realistic Simulation:**
  This setup mirrors how the model would actually be deployed in live trading, being retrained periodically as new data becomes available.

### Comparing Results

Even though the overall precision dropped from approximately **0.85** to **0.68**, the newer result is more trustworthy because:

* The earlier score was based on a single, fixed test set, which could reflect a lucky period rather than consistent predictive power.
* The backtesting result reflects average real-world performance across multiple time periods, making it more robust and less overfitted.


In [71]:
horizons = [2, 5, 55, 220] ## two days, a week, a month, a year worth of trade
new_predictors = []

for horizon in horizons:
    # Create rolling ratio (close vs rolling mean of close)
    ratio_column = f"Close_Ratio_{horizon}"
    df[ratio_column] = df["close"] / df["close"].rolling(horizon).mean()

    # Create rolling trend (sum of past 'target' values)
    trend_column = f"Trend_{horizon}"
    df[trend_column] = df["target"].shift(1).rolling(horizon).sum()

    new_predictors += [ratio_column, trend_column]

# Combine your base features with the new ones
features = [
    'open', 'close', 'min', 'max', 'avg', 'quantity', 'volume',
    'ibovespa_close', 'day_of_week', 'daily_return', 'price_range', 'volume_per_quantity',
    'rolling_close_5', 'rolling_std_5', 'rolling_return_5', 'momentum_5', 'rolling_volume_5'
] + new_predictors

print("Final feature list:")
print(features)


Final feature list:
['open', 'close', 'min', 'max', 'avg', 'quantity', 'volume', 'ibovespa_close', 'day_of_week', 'daily_return', 'price_range', 'volume_per_quantity', 'rolling_close_5', 'rolling_std_5', 'rolling_return_5', 'momentum_5', 'rolling_volume_5', 'Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5', 'Close_Ratio_55', 'Trend_55', 'Close_Ratio_220', 'Trend_220']


### Added Features

Here we added two new features to improve the model's ability to capture both short-term and long-term market patterns:

- **Close_Ratio_h:**
  Measures the relative price position — how far above or below the recent average the price is.
  This helps identify potential overbought or oversold conditions.

- **Trend_h:**
  Captures momentum by counting how many times the price increased in the past *h* sessions.
  This helps the model detect sustained uptrends or downtrends.

In [72]:
df = df.dropna(subset=df.columns[df.columns != "tomorrow"])

In [73]:
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,rolling_volume_5,momentum_5,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_55,Trend_55,Close_Ratio_220,Trend_220
220,2023-05-26,ITUB4,27.40,27.04,26.79,27.76,27.27,29682000,8.096778e+08,110906.0,...,9.474798e+08,0.012962,0.588530,1.0,0.836065,2.0,0.888666,26.0,0.891062,109.0
221,2023-05-26,ABEV3,14.81,14.76,14.64,14.90,14.75,9976800,1.472362e+08,110906.0,...,2.704367e+08,0.008334,0.706220,0.0,0.494572,1.0,0.485010,26.0,0.486266,109.0
222,2023-05-26,BBDC4,16.37,16.12,15.96,16.50,16.16,34702600,5.611164e+08,110906.0,...,9.998005e+08,0.010912,1.044041,1.0,0.580315,2.0,0.533216,26.0,0.531678,109.0
223,2023-05-26,PETR4,26.81,26.80,26.53,26.91,26.75,39866600,1.066492e+09,110906.0,...,1.406994e+09,0.023760,1.248835,2.0,0.895902,3.0,0.909018,26.0,0.883472,110.0
224,2023-05-26,VALE3,66.75,66.33,65.91,67.04,66.57,38181700,2.542137e+09,110906.0,...,2.170333e+09,-0.006352,1.424460,2.0,2.195631,3.0,2.194468,27.0,2.169565,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2023-11-17,ABEV3,13.71,13.60,13.52,13.74,13.56,36788200,4.991464e+08,125062.0,...,3.278229e+08,0.006513,0.932465,0.0,0.402058,2.0,0.419111,25.0,0.433519,108.0
821,2023-11-17,BBDC4,15.57,15.43,15.35,15.60,15.45,34493400,5.331879e+08,125062.0,...,7.900815e+08,0.019693,1.063038,1.0,0.517750,2.0,0.475123,26.0,0.492732,109.0
822,2023-11-17,VALE3,74.23,74.24,74.10,74.65,74.35,24001500,1.784705e+09,125062.0,...,1.985253e+09,0.027885,1.655849,2.0,1.923914,3.0,2.236549,26.0,2.356638,109.0
823,2023-11-17,PETR4,35.83,36.71,35.82,36.92,36.53,85895400,3.138326e+09,125062.0,...,1.865863e+09,0.036888,0.661740,1.0,1.180006,2.0,1.091662,26.0,1.161647,109.0


In [74]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [75]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [76]:
predictions = backtest(df, model, new_predictors)

In [77]:
predictions["Predictions"].value_counts()

Predictions
0.0    416
1.0    139
Name: count, dtype: int64

In [78]:
precision_score(predictions["target"], predictions["Predictions"])

0.8345323741007195

In [79]:
predictions["target"].value_counts() / predictions.shape[0]

target
1    0.522523
0    0.477477
Name: count, dtype: float64

### Backtesting with New Features

After introducing the new features (`Close_Ratio_h` and `Trend_h`) and applying the walk-forward (backtesting) approach, the model achieved a **precision of approximately 0.83**.

- **Precision:** 0.83
  When the model predicts that the price will rise, it is correct about 83% of the time.
  This is a significant improvement over earlier backtesting runs (~0.68 precision), showing that the new features helped the model better capture meaningful market patterns.
  It's important to mention that no evaluation tests were performed here, they'll be displayed in the EvaluationSecondModels.ipynb. Hence we should assume here that the model might (probably) be overfitting due to its high precision.

#### Why Performance Improved

The new features (`Close_Ratio_h` and `Trend_h`) capture both **relative price levels** and **momentum** across multiple time horizons.
This allows the Random Forest model to:

- Recognize **short-term corrections** and **long-term trends**.
- Respond to **multi-scale market dynamics**, improving its ability to generalize.
- Produce more **stable and realistic predictions** when evaluated through walk-forward validation.

Overall, combining richer temporal features with proper backtesting leads to **more reliable and actionable model performance** in a time-series trading context.
